In [1]:
import numpy as np
from coverage import *

import geopandas as gpd
import pandas as pd
import branca
import folium


In [2]:
num_days = 2
tles = gen_sats(sat_nos=[39084,49260])
times = gen_times(start_yr=2021, start_mo=12, start_day=10, days=num_days, step_min=1)

# Generate the instrument from basic specs
# inst = gen_instrument(name="tirs", fl=178, pitch=0.025, h_pix=1850, v_pix=1800, mm=True) # Actual TIRS
inst = gen_instrument(name="tirs", fl=178, pitch=0.025, h_pix=1850, v_pix=4000, mm=True) # Adjusted TIRS for 1 min FOVs

# Select cell size for coverage map
xcell_size = ycell_size = 1

## Select AOI from gpd naturalearth dataset (filter by .name for country, .continent for continent)
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

# aoi =  world[world.name == "Brazil"].geometry
aoi =  world[world.continent == "South America"].geometry
# aoi = world[world.name == "United States of America"]#.geometry

Satellite(s) Loaded from TLE:
[<EarthSatellite LANDSAT 8 catalog #39084 epoch 2022-01-02 20:41:49 UTC>]
[<EarthSatellite LANDSAT 9 catalog #49260 epoch 2022-01-02 16:34:39 UTC>]
Propogation time: 
 2021-12-10 00:00:00+00:00 
to 
 2021-12-11 23:59:00+00:00


In [3]:
# Batch FOV generation over N satellites

gdfs = []
for tle in tles:
    sat = tle[0]
    poly_df = forecast_fovs(sat, times, inst)
    gdfs.append(poly_df)

poly_df = gpd.GeoDataFrame(pd.concat(gdfs, ignore_index=True), crs="epsg:4326")
poly_df["lonspan"] = poly_df.bounds['maxx'] - poly_df.bounds['minx']

# Create cmap for unique satellites and create color column
sat_ids = list(poly_df["id"].unique()).sort()
cmap = branca.colormap.StepColormap(['red', 'blue'], sat_ids, vmin=139084, vmax = 149260)
poly_df['color'] = poly_df['id'].apply(cmap)

In [4]:
# Filter shapes crossing anti-meridian
plot_df = poly_df[poly_df["lonspan"] < 20].copy()

# Filter by AOI
xmin, ymin, xmax, ymax= aoi.total_bounds
plot_df = plot_df.cx[xmin: xmax, ymin: ymax]

m = plot_df.explore(color="color", tooltip=["satellite", "time"])
m#.save("fov_output.html")

In [5]:
## Coverage data analysis for single satellite/ batch of satellites
# 1) Create a grid of equally spaced points

grid, grid_shape = create_grid(aoi.total_bounds, xcell_size, ycell_size)


# 2) Add "n_visits" column to grid using sjoin/ dissolve
shapes = gpd.GeoDataFrame(plot_df.geometry)
merged = gpd.sjoin(shapes, grid, how='left', predicate="intersects")
merged['n_visits']=0 # this will be replaced with nan or positive int where n_visits > 0
dissolve = merged.dissolve(by="index_right", aggfunc="count") # no difference in count vs. sum here?
grid.loc[dissolve.index, 'n_visits'] = dissolve.n_visits.values

grid.n_visits.describe()


count    961.000000
mean       1.144641
std        0.363569
min        1.000000
25%        1.000000
50%        1.000000
75%        1.000000
max        3.000000
Name: n_visits, dtype: float64

In [6]:
# Plotting
# Form 2D array of n_visits based on grid shape
img = np.rot90(grid.n_visits.values.reshape(grid_shape))

# Create colormap and apply to img
colormap = branca.colormap.step.viridis.scale(1, grid.n_visits.max())
# colormap = branca.colormap.step.viridis.scale(1, 2)

def colorfunc(x):
    if np.isnan(x):
        return (0,0,0,0)
    else:
        return colormap.rgba_bytes_tuple(x)

# Apply cmap to img array and rearrange for RGBA
cmap = np.vectorize(colorfunc)
rgba_img = np.array(cmap(img))
rgba_img = np.moveaxis(rgba_img, 0, 2)

# Update image corner bounds based on cell size
xmin, ymin, xmax, ymax= grid.total_bounds
xmin = xmin - xcell_size/2
ymin = ymin - ycell_size/2
xmax = xmax + xcell_size/2
ymax = ymax + ycell_size/2

m = folium.Map()
m.fit_bounds([[ymin, xmin], [ymax, xmax]])
m.add_child(folium.raster_layers.ImageOverlay(rgba_img, opacity=.4, mercator_project=True,# crs="EPSG:4326",
                                 bounds = [[ymin,xmin],[ymax,xmax]]))
colormap.add_to(m)
m#.save("coverage_output.html")


In [7]:
print(branca.__version__)

0.4.2


In [8]:
import rasterio
from rasterio.transform import Affine

In [10]:

x = np.linspace(-4.0, 4.0, 240)
y = np.linspace(-3.0, 3.0, 180)
X, Y = np.meshgrid(x, y)
Z1 = np.exp(-2 * np.log(2) * ((X - 0.5) ** 2 + (Y - 0.5) ** 2) / 1 ** 2)
Z2 = np.exp(-3 * np.log(2) * ((X + 0.5) ** 2 + (Y + 0.5) ** 2) / 2.5 ** 2)
Z = 10.0 * (Z2 - Z1)
res = (x[-1] - x[0]) / 240.0

In [11]:
transform = Affine.translation(x[0] - res / 2, y[0] - res / 2) * Affine.scale(res, res)

In [12]:
with rasterio.open(
    './tmp/new.tif',
    'w',
    driver='GTiff',
    height=Z.shape[0],
    width=Z.shape[1],
    count=1,
    dtype=Z.dtype,
    crs='+proj=latlong',
    transform=transform,
) as dst:
    dst.write(Z, 1)

dst.close()


In [13]:
dataset = rasterio.open('./tmp/new.tif')

In [14]:
# dataset
dataset.bounds
# dataset.crs

BoundingBox(left=-4.016666666666667, bottom=2.9833333333333334, right=3.9833333333333334, top=-3.0166666666666666)

In [15]:
dataset.read(1)

array([[0.02122529, 0.02293761, 0.0247696 , ..., 0.00180847, 0.00163782,
        0.00148217],
       [0.02243408, 0.02424392, 0.02618023, ..., 0.00191146, 0.00173109,
        0.00156658],
       [0.02369399, 0.02560546, 0.02765053, ..., 0.00201881, 0.00182831,
        0.00165456],
       ...,
       [0.00336547, 0.00363697, 0.00392745, ..., 0.00028675, 0.00025969,
        0.00023501],
       [0.00311622, 0.00336762, 0.00363659, ..., 0.00026551, 0.00024046,
        0.00021761],
       [0.00288328, 0.00311589, 0.00336475, ..., 0.00024567, 0.00022248,
        0.00020134]])

In [16]:
import plotly.express as px

In [17]:
px.imshow(dataset.read(1))